As linhas de código fornecidas importam bibliotecas necessárias, definem diretórios de imagens e listam os arquivos presentes nesses diretórios.

In [1]:
import numpy as np # linear algebra
import os, sys
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import cv2
import matplotlib.pyplot as plt
import scipy.stats
import tensorflow as tf

from sklearn.model_selection import train_test_split

from keras import applications, optimizers, Input
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

2024-03-18 12:17:24.377219: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 12:17:24.477495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 12:17:24.477534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 12:17:24.479121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 12:17:24.489716: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 12:17:24.490478: I tensorflow/core/platform/cpu_feature_guard.cc:1

O código acima está dividindo um conjunto de imagens em conjuntos de treinamento e validação.

In [2]:
folder = 'coins/br-coins/circles'

image_width = 120
image_height = 120
channels = 3

train_files = []
i=0
for coin in [5, 10, 25, 50, 100]:
    onlyfiles = [f for f in os.listdir(folder + '/' + str(coin)) if os.path.isfile(os.path.join(folder + '/' + str(coin), f))]
    for _file in onlyfiles:
        train_files.append(_file)

dataset = np.ndarray(shape=(len(train_files), image_height, image_width, channels),
                     dtype=np.float32)
y_dataset = []

i = 0
for coin in [5, 10, 25, 50, 100]:
    onlyfiles = [f for f in os.listdir(folder + '/' + str(coin)) if os.path.isfile(os.path.join(folder + '/' + str(coin), f))]
    for _file in onlyfiles:
        #img = load_img(folder + '/' +  str(coin) + '/' + _file)  # this is a PIL image
        #img.thumbnail((image_width, image_height))
        # Convert to Numpy Array
        #x = img_to_array(img)
        # open with opencv
        x = np.array(cv2.imread(folder + '/' +  str(coin) + '/' + _file))
        #x = x.reshape((3, 120, 160))
        # Normalize
        #x = (x - 128.0) / 128.0

        dataset[i] = x
        mapping = {5: 0, 10: 1, 25: 2, 50: 3, 100: 4}
        y_dataset.append(mapping[coin])
        i += 1
        if i % 250 == 0:
            print("%d images to array" % i)
print("All images to array!")

O trecho de código fornecido cria diretórios para armazenar dados de treinamento e validação, e em seguida, copia imagens para esses diretórios.

In [3]:
dataset[0].shape

O trecho de código abaixo está relacionado ao pré-processamento de dados de treinamento e validação para um modelo de aprendizado de máquina que lida com imagens. 

In [4]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)

train_dataset = train_data_gen.flow_from_directory(
    os.path.join(BASE_DIR, train_dir),
    target_size = (227,227),
    class_mode = 'categorical'
)

val_dataset = val_data_gen.flow_from_directory(
    os.path.join(BASE_DIR, val_dir),
    target_size = (227,227),
    class_mode = 'categorical'
)


Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


O código acima define um modelo de rede neural convolucional (CNN) usando a biblioteca TensorFlow.

In [5]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(227, 227, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
#model.add(tf.keras.layers.Dropout(0.25))

#model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation='softmax'))



2024-03-18 12:17:31.612530: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-03-18 12:17:31.682572: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-03-18 12:17:31.725823: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


O código fornecido mostra uma sequência de operações relacionadas à configuração e compilação de um modelo de rede neural usando a biblioteca TensorFlow.

In [6]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 401408)            0         
                                                                 
 dense (Dense)               (None, 64)                25690176  
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
 flatten_1 (Flatten)         (None, 3)                 0         
                                                        

Compilando o modelo de rede neural usando a biblioteca TensorFlow

In [7]:
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 15)
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset)


Epoch 1/5


2024-03-18 12:17:33.516139: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19787136 exceeds 10% of free system memory.
2024-03-18 12:17:34.027167: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


8/8 [==============================] - 22s 2s/step - loss: 1.0818 - accuracy: 0.2875 - val_loss: 1.0663 - val_accuracy: 0.3000
Epoch 2/5
1/8 [==>...........................] - ETA: 17s - loss: 1.0675 - accuracy: 0.2500

KeyboardInterrupt: 

O trecho de código abaixo está usando a biblioteca matplotlib.pyplot para criar um gráfico da função de perda de um modelo de treinamento

In [ ]:

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.title("Função de perda")
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend(["Treinando"], loc='upper left')
plt.savefig('my_figure.png')

O código acima está salvando um modelo treinado em um arquivo chamado "model_train_separado.h5".

In [ ]:
model.save(os.path.join(BASE_DIR,'model_train_separado.h5'))

/home/emanuel/Área de Trabalho/UFERSA-2023.2/PAPAYA/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Este trecho de código realiza a avaliação de um modelo de aprendizado de máquina utilizando métricas de desempenho. 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Fazendo previsões no conjunto de validação
predictions = model.predict(val_dataset)
predicted_classes = np.argmax(predictions, axis=1)

# Obtendo as classes verdadeiras do conjunto de validação
true_classes = val_dataset.classes
print("P Classes", predicted_classes)
print("T Classes ", true_classes)

# Calculando as métricas de avaliação
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Sensibilidade:", recall)
print("F1-Score:", f1)

2/2 [==============================] - 2s 847ms/step
P Classes [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
T Classes  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Acurácia: 0.3333333333333333
Precisão: 0.1111111111111111
Sensibilidade: 0.3333333333333333
F1-Score: 0.16666666666666666


/home/emanuel/Área de Trabalho/UFERSA-2023.2/PAPAYA/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


A matriz de confusão é uma ferramenta comumente usada para avaliar o desempenho de um modelo de classificação. Ela mostra a quantidade de acertos e erros de classificação para cada classe do modelo.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(true_classes, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
disp.ax_.set_title('Matriz de Confusão')
disp.ax_.set_xlabel('Classificação Prevista')
disp.ax_.set_ylabel('Classificação Real')
plt.savefig('confusion_matrix.png')
#plt.show()